In [ ]:
import pandas as pd

change = {"-":"","+":"","in": "", "- -":""," ":""}
df = pd.read_csv('./raw_data/koran/KC_AMBON/Desember/3. Rekening Koran_604_APBN_122024/combined_transactions.csv')
for old, new in change.items():
    df["Tanggal Transaksi"] = df["Tanggal Transaksi"].astype(str).apply(lambda x: x.replace(old,new))
df.to_csv('output_apbn_kotor.csv')


In [ ]:
import pandas as pd
import re
import os

df_list = []
for dirpath, dirs, files in os.walk('./raw_data/koran/KC_BONE/Oktober/3. Rekening Koran_611_APBN_102024'):
    # Sort files numerically by extracting the number from the file name
    sorted_files = sorted(files, key=lambda x: int(re.search(r'\d+', x).group()) if re.search(r'\d+', x) else float('inf'))
    
    for file in sorted_files:
        if file.endswith(".csv"):
            df = pd.read_csv(os.path.join(dirpath, file), dtype='object')
            keywords = ["Nama Produk", "Valuta", "Alamat Unit Kerja", "Periode Transaksi", "No. Rekening", "Unit Kerja",
                        "Tanggal Transaksi", "Saldo Awal", "0.00","NaN","nan","500,000.00"]
            column_names = ["tanggal_transaksi", "uraian_transaksi", "teller", "debet", "kredit", "saldo"]
            change = {"-":"","+":"","in": "","- -":"","- .. ":""}

            for old, new in change.items():
                df["Tanggal Transaksi"] = df["Tanggal Transaksi"].astype(str).apply(lambda x: x.replace(old,new))
            # Filter out rows containing keywords in 'Tanggal Transaksi' column
            df = df[~df["Tanggal Transaksi"].astype(str).str.contains("|".join(keywords), na=True)]
            df.columns = column_names
            df.to_csv('output_apbn_kotor.csv',index=False)
            
            # Extract valid dates using regex and convert to datetime
            df["tanggal_transaksi"] = [
            re.search(r"\d{2}/\d{2}/\d{2}", str(item)).group() 
            if re.search(r"\d{2}/\d{2}/\d{2}", str(item)) 
            else None 
            for item in df["tanggal_transaksi"]]

            df.reset_index(drop=True, inplace=True)
        
            # Convert to datetime format
            df["tanggal_transaksi"] = pd.to_datetime(df["tanggal_transaksi"], format='%d/%m/%y', errors='coerce').dt.date
            
            # Convert columns to integers
            int_cols = ["debet", "kredit", "saldo"]
            for kolom in int_cols:
                df[kolom] = df[kolom].replace({',': ''}, regex=True)  # Remove commas
                df[kolom] = df[kolom].str.split('.').str[0]  # Remove decimal part
                df[kolom] = pd.to_numeric(df[kolom], errors='coerce').astype('Int64')
            
            # Add additional columns with fixed values
            df["jenis"] = "TASPEN THT"
            df["no_rekening"] = "000101000526307"
            df["nama_produk"] = "Giro Umum-IDR"
            df["valuta"] = "IDR"
            df["tanggal_laporan"] = pd.to_datetime("02/01/25", format='%d/%m/%y', errors='coerce').date()
            df["unit_kerja"] = "KC Ambon"
            df["alamat_unit_kerja"] = "Jl. Diponegoro No. 29 Ambon"
            
            df_list.append(df)

# Concatenate all dataframes and save to Excel
output_path = './cleaned_data/koran/KC_AMBON/Desember'
if not os.path.exists(output_path):
    os.makedirs('./cleaned_data/koran/KC_AMBON/Desember', exist_ok=True) 
df = pd.concat(df_list, axis=0)
df.to_excel(os.path.join(output_path, "Output_THT.xlsx"), index=False)
print(f"Data has been cleaned and saved to {output_path}")
print(f'data berjumlah {len(df)} baris')
del df_list

In [ ]:
import pandas as pd
import re
import os

df_list = []
for dirpath, dirs, files in os.walk('./raw_data/koran/KC_BONE/Oktober/3. Rekening Koran_611_APBN_102024'):
    # Sort files numerically by extracting the number from the file name
    sorted_files = sorted(files, key=lambda x: int(re.search(r'\d+', x).group()) if re.search(r'\d+', x) else float('inf'))
    
    for file in sorted_files:
        if file.endswith(".csv"):
            df = pd.read_csv(os.path.join(dirpath, file), dtype='object')
            keywords = ["nan","NaN","OPENING BALANCE","1,306.00","477,441.00","500,000.00","1,241.00","0.00","Period :","Today Hold Balance :","Account Status :","Account Name :","Account No :","DATE", "2,129,536.50"]
            column_names = ["date","time", "remark", "debet", "credit","ledger", "teller_id"]
            df.columns = column_names
            df.drop("time", axis=1, inplace=True)
            
            # Filter out rows containing keywords in 'Tanggal Transaksi' column
            df = df[~df["date"].astype(str).str.contains("|".join(keywords), na=False)]
            
            # Extract valid dates using regex and convert to datetime
            df["date"] = [
            re.search(r"\d{2}/\d{2}/\d{2}", str(item)).group() 
            if re.search(r"\d{2}/\d{2}/\d{2}", str(item)) 
            else None 
            for item in df["date"]]
            df.reset_index(drop=True, inplace=True)
            
            # Convert to datetime format
            df["date"] = pd.to_datetime(df["date"], format='%d/%m/%y', errors='coerce').dt.date
            
            # Convert columns to integers
            int_cols = ["debet", "credit","ledger"]
            for kolom in int_cols:
                df[kolom] = df[kolom].replace({',': ''}, regex=True)  # Remove commas
                df[kolom] = df[kolom].str.split('.').str[0]  # Remove decimal part
                df[kolom] = pd.to_numeric(df[kolom], errors='coerce').astype('Int64')
            
            # Add additional columns with fixed values
            df["account_no"] = "0111-01-002639-30-1"
            df["account_name"] = "TASPEN PROGRAM PE"
            df["account_status"] = "Active"
            df["today_hold_balance"] = 0
            df["branch"] = "KC Bone"
            
            df_list.append(df)

# Concatenate all dataframes and save to Excel
output_path = './cleaned_data/koran/KC_BONE/Oktober'
if not os.path.exists(output_path):
    os.makedirs(output_path)
df = pd.concat(df_list, axis=0)
df.to_excel(os.path.join(output_path,"Output_APBN.xlsx"), index=False)
print(f"Data has been cleaned and saved to {output_path}")
print(f'data berjumlah {len(df)} baris')
del df_list

<h2>Without Ledger</h2>

In [46]:
import pandas as pd
import re
import os

df_list = []
for dirpath, dirs, files in os.walk('./raw_data/koran/KC_BONE/Oktober/3. Rekening Koran_611_APBN_102024'):
    # Sort files numerically by extracting the number from the file name
    sorted_files = sorted(files, key=lambda x: int(re.search(r'\d+', x).group()) if re.search(r'\d+', x) else float('inf'))
    
    for file in sorted_files:
        if file.endswith(".csv"):
            df = pd.read_csv(os.path.join(dirpath, file), dtype='object')
            keywords = ["1,492,472.50","nan","NaN","OPENING BALANCE","477,441.00","500,000.00","0.00","Period :","Today Hold Balance :","Account Status :","Account Name :","Account No :","DATE", "2,129,536.50"]
            column_names = ["date","time", "remark", "debet", "credit", "teller_id"]
            df.columns = column_names
            df.drop("time", axis=1, inplace=True)
            
            # Filter out rows containing keywords in 'Tanggal Transaksi' column
            df = df[~df["date"].astype(str).str.contains("|".join(keywords), na=False)]
            
            # Extract valid dates using regex and convert to datetime
            df["date"] = [
            re.search(r"\d{2}/\d{2}/\d{2}", str(item)).group() 
            if re.search(r"\d{2}/\d{2}/\d{2}", str(item)) 
            else None 
            for item in df["date"]]
            df.reset_index(drop=True, inplace=True)
            
            # Convert to datetime format
            df["date"] = pd.to_datetime(df["date"], format='%d/%m/%y', errors='coerce').dt.date
            
            # Convert columns to integers
            int_cols = ["debet", "credit"]
            for kolom in int_cols:
                df[kolom] = df[kolom].replace({',': ''}, regex=True)  # Remove commas
                df[kolom] = df[kolom].str.split('.').str[0]  # Remove decimal part
                df[kolom] = pd.to_numeric(df[kolom], errors='coerce').astype('Int64')
            
            # Add additional columns with fixed values
            df["account_no"] = "0111-01-002639-30-1"
            df["account_name"] = "TASPEN PROGRAM PE"
            df["account_status"] = "Active"
            df["today_hold_balance"] = 0
            df["branch"] = "KC Bone"
            
            df_list.append(df)

# Concatenate all dataframes and save to Excel
output_path = './cleaned_data/koran/KC_BONE/Oktober'
if not os.path.exists(output_path):
    os.makedirs(output_path)
df = pd.concat(df_list, axis=0)
df.to_excel(os.path.join(output_path,"Output_APBN.xlsx"), index=False)
print(f"Data has been cleaned and saved to {output_path}")
print(f'data berjumlah {len(df)} baris')
del df_list

Data has been cleaned and saved to ./cleaned_data/koran/KC_BONE/Oktober
data berjumlah 176 baris
